In [1]:
import pdfplumber
import pandas as pd
import re

# Percorso del file PDF AGCOM
file_pdf = r"C:\Users\Antonio Maurizi\Desktop\PROGETTO ESAME EPICODE\RapportoAggiornamentoBBmap_2Q25vs4Q24_r250716 (1).pdf"

#Lista operatori noti presenti nel PDF

operatori_noti = [
    "4All",
    "Air2bite",
    "Alto Adige Reti",
    "Atomo Networks",
    "Bbanda",
    "Connesi",
    "Dodonet",
    "Eolo",
    "Estracom",
    "Fastweb",
    "Fibercop",
    "Fibreconnect",
    "Fibersurf",
    "Intred",
    "Iride",
    "Isiline",
    "Lepida",
    "Lenfiber",
    "Linkem",
    "Micro Servizi",
    "Mynet",
    "NetValue",
    "Open Fiber",
    "Planetel",
    "Revos",
    "Sicilink Internet On Air",
    "Sinet",
    "Spadhausen",
    "Spadhausen Internet Provider",
    "Telweb",
    "Terrecablate",
    "Tim",
    "Unidata",
    "Valcom",
    "Valcom Calabria"
]


# Funzione semplice che ripulisce il testo (elimina spazi inutili e simboli strani)
def pulisci_testo(testo):
    if not isinstance(testo, str):
        return testo
    testo = testo.replace("Â", "").replace("•", "• ").strip()
    testo = re.sub(r"\s{2,}", " ", testo)
    return testo

# Funzione che ripulisce i nomi degli operatori (toglie sigle come SRL, SPA)
def pulisci_operatore(op):
    op = re.sub(r"\b(S\.?R\.?L\.?|S\.?P\.?A\.?)\b", "", op, flags=re.IGNORECASE)
    return re.sub(r"\s{2,}", " ", op).strip()

# Liste per salvare i dati trovati
dati = []
regione = ""
provincia = ""

# Lettura del PDF
with pdfplumber.open(file_pdf) as pdf:
# Scorro le pagine con il ciclo for partendo dalla pagina 259 deputata ai dati essenziali
    for pagina in pdf.pages[258:]:  
        testo = pagina.extract_text()
        if not testo:
            continue

        righe = testo.split("\n")

        for riga in righe:
            riga = pulisci_testo(riga)
            if not riga:
                continue

            # Controllo se la riga indica il nome della Regione
            match_regione = re.match(r"^\d+\.\d+\.\s+(.+)$", riga)
            if match_regione:
                regione = match_regione.group(1).strip()
                continue

            # Controllo se appare la Provincia
            match_provincia = re.search(r"provincia di ([A-Za-zÀ-ÖØ-öø-ÿ' ]+)\.?", riga, re.IGNORECASE)
            if match_provincia:
                provincia = match_provincia.group(1).strip()
                continue

            #   Se la riga elenca dei comuni (sono preceduti dal simbolo •) e verifico gli Op Noti elencati
            # inoltre tolgo il nome dell’operatore dal testo per isolare il Comune
            if "•" in riga:
                parte = riga.split("•")[-1].strip()
                operatori_trovati = [op for op in operatori_noti if op.lower() in parte.lower()]

                if operatori_trovati:
                    comune = parte
                    for op in operatori_trovati:
                        comune = re.sub(op, "", comune, flags=re.IGNORECASE)
                    comune = pulisci_operatore(comune).strip(" ;,")
                    operatori_trovati = [pulisci_operatore(op) for op in operatori_trovati]

                    for op in operatori_trovati:
                        dati.append({
                            "Regione": regione,
                            "Provincia": provincia,
                            "Comune": comune,
                            "Operatore": op
                        })

# Creazione del DataFrame finale
network_operator = pd.DataFrame(dati)
network_operator.drop_duplicates(inplace=True)
network_operator = network_operator[(network_operator["Comune"] != "") & (network_operator["Operatore"] != "")]

# Pulizia generale
for col in ["Regione", "Provincia", "Comune", "Operatore"]:
    network_operator[col] = network_operator[col].astype(str).str.strip()
    
print(network_operator)

       Regione Provincia      Comune   Operatore
0      Abruzzo    Chieti      Altino    Fibercop
1      Abruzzo    Chieti       Archi    Fibercop
2      Abruzzo    Chieti       Archi  Open Fiber
3      Abruzzo    Chieti         Ari    Fibercop
4      Abruzzo    Chieti         Ari  Open Fiber
...        ...       ...         ...         ...
16029   Veneto   Vicenza  Zermeghedo    Fibercop
16030   Veneto   Vicenza  Zermeghedo  Open Fiber
16031   Veneto   Vicenza   Zovencedo    Fibercop
16032   Veneto   Vicenza    Zugliano    Fibercop
16033   Veneto   Vicenza    Zugliano  Open Fiber

[16033 rows x 4 columns]
